In [1]:
import os

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/2.part/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Part Number'].tolist()

总数量：507


['30250S',
 '30250SX',
 '30420S',
 '30430S',
 '30460S',
 '30460SA',
 '30460SB',
 '30470S',
 '30470SX',
 '30490S',
 '30490SA',
 '30490SAX',
 '30490SX',
 '30719S',
 '30719SHWP',
 '31007S',
 '31035S',
 '31035SX',
 '31036S',
 '31036SA',
 '31038S',
 '31038SX',
 '31041S',
 '31045S',
 '31045SA',
 '31045SX',
 '31047S',
 '31047SH',
 '31047SHWP',
 '31047SK3',
 '31047SVVT',
 '31047SX',
 '31047SXH',
 '31047SXHWP',
 '31049S',
 '31049SA',
 '31049SAH',
 '31049SAVVT1',
 '31049SAW',
 '31049SAWVVT',
 '31049SAWVVT1',
 '31049SB',
 '31049SBH',
 '31049SBX',
 '31049SBXH',
 '31049SC',
 '31049SCH',
 '31049SCX',
 '31049SCXH',
 '31049SD',
 '31049SDH',
 '31049SDX',
 '31049SDXH',
 '31049SE',
 '31049SEH',
 '31049SEK3',
 '31049SEVVT',
 '31049SF',
 '31049SFH',
 '31049SFVVT',
 '31049SG',
 '31049SGX',
 '31049SH',
 '31049SJ',
 '31049SK',
 '31049SL',
 '31049SM',
 '31049SN',
 '31049SP',
 '31049SPVVT',
 '31049SVVT1',
 '31049SVVT2',
 '31049SW',
 '31051S',
 '31051SA',
 '31051SAX',
 '31051SX',
 '31054S',
 '31055S',
 '31055SX'

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

payload = {'partData': {'listing_data_essential': {'parttype': input_.loc[a, 'Type Code'],
                                                   'partkey': input_.loc[a, 'Key']},
                        'listing_data_supplemental': {'partnumber': input_.loc[a, 'Part Number'],
                                                      'catalogname': input_.loc[a, 'Manufacturer']}}}

data = {'func': 'getbuyersguide',
        'payload': json.dumps(payload),
        'api_json_request': '1'}

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                             data=data,
                             headers=get_header(),
                             proxies=get_proxy(),
                             timeout=(10, 10))

        if resp.status_code == 200:
            json_ = resp.json()
    
            if 'redirect_to_url' not in json_ and json_['buyersguidepieces']['title'].startswith('Buyer\'s Guide :') and 'collected_javascript' in json_:
                break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1


<Response [200]>

In [8]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(json_['buyersguidepieces']['body'], 'lxml')
html = etree.HTML(str(soup))

with open('./vehicle.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="buyersguide-nested">
   <div style="padding: .5em;">
    <table class="nobmp" style="width: 100%;">
     <tr>
      <td class="altrow-a-0">
       MAZDA
      </td>
      <td class="altrow-a-0">
       3
      </td>
      <td class="altrow-a-0">
       2004-2007
      </td>
     </tr>
     <tr>
      <td class="altrow-a-1">
       MAZDA
      </td>
      <td class="altrow-a-1">
       5
      </td>
      <td class="altrow-a-1">
       2006-2007
      </td>
     </tr>
     <tr>
      <td class="altrow-a-0">
       MAZDA
      </td>
      <td class="altrow-a-0">
       6
      </td>
      <td class="altrow-a-0">
       2003-2007
      </td>
     </tr>
    </table>
    <div>
     Please refer to catalog for application details.
    </div>
   </div>
  </div>
 </body>
</html>



In [9]:
dict_vehicle = {}
len(html.xpath('//p/text()')) != 0 and html.xpath('//p/text()')[0] == 'No applications found.'

False

In [10]:
list_tr = html.xpath('//tr')

list_tr

[<Element tr at 0x12bdf58c0>,
 <Element tr at 0x12bdf52c0>,
 <Element tr at 0x12bdf53c0>]

In [11]:
len(list_tr) == 0

False

In [12]:
for i, tr in zip(range(len(list_tr)), list_tr):
    dict_vehicle[str(i)] = {}
    list_td = [text.strip() for text in tr.xpath('./td/text()')]
    for j, td in zip(range(len(list_td)), list_td):
        dict_vehicle[str(i)][str(j)] = td

dict_vehicle

{'0': {'0': 'MAZDA', '1': '3', '2': '2004-2007'},
 '1': {'0': 'MAZDA', '1': '5', '2': '2006-2007'},
 '2': {'0': 'MAZDA', '1': '6', '2': '2003-2007'}}

In [13]:
df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Group': input_.loc[a, 'Group'],
                         'Type': input_.loc[a, 'Type'],
                         'Type Code': input_.loc[a, 'Type Code'],
                         'Manufacturer': input_.loc[a, 'Manufacturer'],
                         'Part Number': input_.loc[a, 'Part Number'],
                         'Parameter': input_.loc[a, 'Parameter'],
                         'Json_Price': input_.loc[a, 'Json_Price'],
                         'Note 1': input_.loc[a, 'Note 1'],
                         'Note 2': input_.loc[a, 'Note 2'],
                         'Json_Vehicle': json.dumps(dict_vehicle),
                         'OEM': input_.loc[a, 'OEM'],
                         'Picture': input_.loc[a, 'Picture'],
                         'Url': input_.loc[a, 'Url'],
                         'Info': input_.loc[a, 'Info'],
                         'Json_Src': input_.loc[a, 'Json_Src'],
                         'Key': input_.loc[a, 'Key']}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Group,Type,Type Code,Manufacturer,Part Number,Parameter,Json_Price,Note 1,Note 2,Json_Vehicle,OEM,Picture,Url,Info,Json_Src,Key
0,1,Engine,Timing Chain & Component Kit,5756,MELLING,30250S,,"{""0"": ""$172.79""}",,Category: Timing Chain &amp; Component Kit,"{""0"": {""0"": ""MAZDA"", ""1"": ""3"", ""2"": ""2004-2007...",,,"https://www.rockauto.com/en/parts/melling,3025...",https://www.rockauto.com/en/moreinfo.php?pk=83...,"{""0"": ""https://www.rockauto.com/info/583/3-025...",8304960


In [14]:
crawler_status = 'ok'

crawler_status

'ok'

In [15]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [16]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_vehicle.xlsx', index=False)

output_correct

,No,Group,Type,Type Code,Manufacturer,Part Number,Parameter,Json_Price,Note 1,Note 2,Json_Vehicle,OEM,Picture,Url,Info,Json_Src,Key
0,1,Engine,Timing Chain & Component Kit,5756,MELLING,30250S,,"{""0"": ""$172.79""}",,Category: Timing Chain &amp; Component Kit,"{""0"": {""0"": ""MAZDA"", ""1"": ""3"", ""2"": ""2004-2007...",,,"https://www.rockauto.com/en/parts/melling,3025...",https://www.rockauto.com/en/moreinfo.php?pk=83...,"{""0"": ""https://www.rockauto.com/info/583/3-025...",8304960


In [17]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_vehicle_error.xlsx', index=False)

output_error

""
